In [ ]:
#chmom：月频率。 从 t − 6 到 t − 1 减去 t − 12 到 t − 7 的累积回报。
#maxret：月频率。在第 t-1月期间的最大日回报率
#mom1m：月频率。单月累计回报。
#mom6m：月频率。在月底前一个月结束的5个月累计收益。从 t − 6 到 t − 1
#mom12m：月频率。在月底前一个月结束的 11个月累计收益。从 t − 12 到 t − 1
#mom36m：月频率。从 t-36月到 t-13月的累计收益。

In [1]:
import numpy as np
import pandas as pd
import os
from itertools import product

In [2]:
mon_lst = []
for y in range(2000, 2025):
    for m in ['01-31', '02-29' if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else '02-28', '03-31', '04-30', '05-31', '06-30', '07-31', '08-31', '09-30', '10-31', '11-30', '12-31']:
        mon_lst.append(f'{y}-{m}')
mon_df = pd.DataFrame(mon_lst[:-2], columns=['交易月份'])
mon_df['月份'] = mon_df['交易月份'].apply(lambda x: x[:7])
stk_df = pd.read_csv('公司文件.csv')
stk_df['on'] = 0
mon_df['on'] = 0
header = pd.merge(stk_df, mon_df, on=['on'], how='left')[['证券代码', '交易月份', '月份']]
header

,证券代码,交易月份,月份
0,1,2000-01-31,2000-01
1,1,2000-02-29,2000-02
2,1,2000-03-31,2000-03
3,1,2000-04-30,2000-04
4,1,2000-05-31,2000-05
...,...,...,...
1719157,920118,2024-06-30,2024-06
1719158,920118,2024-07-31,2024-07
1719159,920118,2024-08-31,2024-08
1719160,920118,2024-09-30,2024-09


In [4]:
import numpy as np
import pandas as pd
import warnings
from itertools import product

# 忽略警告
warnings.filterwarnings("ignore")

# 读取数据
RET = pd.read_csv('日_日个股回报率文件.csv', usecols=['证券代码', '交易日期', '不考虑现金红利的日个股回报率'])
RET = RET.rename(columns={'不考虑现金红利的日个股回报率': 'RET'})
RET['Trddt'] = pd.to_datetime(RET['交易日期'], format='%Y-%m-%d')
RET['year'] = RET['Trddt'].dt.year
RET['month'] = RET['Trddt'].dt.month
RET['DATE'] = RET['Trddt'].astype(str).replace('\-', '', regex=True)
RET['Yearmon'] = RET['DATE'].astype(int) // 100

# 月份编号
Mon_list = np.unique(RET['Yearmon'])
R2 = pd.DataFrame(columns=['Yearmon', 'ID_Mon'], index=np.arange(len(Mon_list)))
R2['Yearmon'] = Mon_list
R2['ID_Mon'] = np.arange(len(Mon_list))
RET_ = pd.merge(RET, R2, on='Yearmon', how='left')

Stkcd_list = np.unique(RET['证券代码'])

# 构建股票-年月表格
X1 = []
X2 = []
for x1, x2 in product(Stkcd_list, Mon_list):
    X1.append(x1)
    X2.append(x2)

R1 = pd.DataFrame(columns=['证券代码', 'Yearmon'], index=np.arange(len(X1)))
R1['证券代码'] = X1
R1['Yearmon'] = X2
R1 = pd.merge(R1, R2, on='Yearmon', how='left')

R1['ID_Mon_m1'] = R1['ID_Mon'] - 1
maxret = RET_.groupby(['证券代码', 'ID_Mon'])['RET'].max().reset_index()
maxret = maxret.rename(columns={'RET': 'maxret'})

R1 = pd.merge(R1, maxret,
              left_on=['ID_Mon_m1', '证券代码'],
              right_on=['ID_Mon', '证券代码'],
              how='left')
del R1['ID_Mon_y']
R1 = R1.rename(columns={'ID_Mon_x': 'ID_Mon'})

RETM = RET_.groupby(['证券代码', 'ID_Mon'])['RET'].sum().reset_index()
RETM = RETM.rename(columns={'RET': 'mom1m'})

R1 = pd.merge(R1, RETM,
              left_on=['ID_Mon', '证券代码'],
              right_on=['ID_Mon', '证券代码'],
              how='left')

# 计算 mom6m
M6 = RETM.groupby(['证券代码'])['mom1m'].rolling(window=5).sum().reset_index()
M6 = M6.rename(columns={'mom1m': 'mom6m'})
del M6['level_1']
M6_ = pd.concat([M6, RETM['ID_Mon']], axis=1)
R1 = pd.merge(R1, M6_,
              left_on=['ID_Mon_m1', '证券代码'],
              right_on=['ID_Mon', '证券代码'],
              how='left')
del R1['ID_Mon_y']
R1 = R1.rename(columns={'ID_Mon_x': 'ID_Mon'})

# 计算 mom12m
M12 = RETM.groupby(['证券代码'])['mom1m'].rolling(window=11).sum().reset_index()
M12 = M12.rename(columns={'mom1m': 'mom12m'})
del M12['level_1']
M12_ = pd.concat([M12, RETM['ID_Mon']], axis=1)
R1 = pd.merge(R1, M12_,
              left_on=['ID_Mon_m1', '证券代码'],
              right_on=['ID_Mon', '证券代码'],
              how='left')
del R1['ID_Mon_y']
R1 = R1.rename(columns={'ID_Mon_x': 'ID_Mon'})

# 计算 T36
T36 = RETM.groupby(['证券代码'])['mom1m'].rolling(window=36).sum().reset_index()
T36 = T36.rename(columns={'mom1m': 'T36'})
del T36['level_1']
T36_ = pd.concat([T36, RETM['ID_Mon']], axis=1)
R1 = pd.merge(R1, T36_,
              left_on=['ID_Mon_m1', '证券代码'],
              right_on=['ID_Mon', '证券代码'],
              how='left')
del R1['ID_Mon_y']
R1 = R1.rename(columns={'ID_Mon_x': 'ID_Mon'})

# 计算 T12
T12 = RETM.groupby(['证券代码'])['mom1m'].rolling(window=12).sum().reset_index()
T12 = T12.rename(columns={'mom1m': 'T12'})
del T12['level_1']
T12_ = pd.concat([T12, RETM['ID_Mon']], axis=1)
R1 = pd.merge(R1, T12_,
              left_on=['ID_Mon_m1', '证券代码'],
              right_on=['ID_Mon', '证券代码'],
              how='left')
del R1['ID_Mon_y']
R1 = R1.rename(columns={'ID_Mon_x': 'ID_Mon'})

# 计算动量
R1['mom36m'] = R1['T36'] - R1['T12']

# 计算 chmom 从 t-6 到 t-1 减去从 t-12 到 t-7 的累计回报
R1['chmom'] = R1['mom6m'] - (R1['mom12m'] - R1['mom6m'])

# 选择需要的列并转换 Yearmon
R1_ = R1[['证券代码', 'Yearmon', 'maxret', 'mom1m', 'mom6m', 'mom12m', 'mom36m','chmom']]
R1_['月份'] = R1_['Yearmon'].astype(str).str[:4] + '-' + R1_['Yearmon'].astype(str).str[4:6]

# 最终选择的列
R1_ = R1_[['证券代码', '月份', 'chmom']]

# 假设 header 是一个已有的数据框
df = pd.merge(header, R1_, on=['证券代码', '月份'], how='left')

# 输出到 CSV
df[['证券代码', '交易月份', 'chmom']].to_csv('月_18.csv', index=False)

In [6]:
df1 = pd.read_csv('月_18.csv')
df1

,证券代码,交易月份,chmom
0,1,2000-01-31,NaN
1,1,2000-02-29,NaN
2,1,2000-03-31,NaN
3,1,2000-04-30,NaN
4,1,2000-05-31,NaN
...,...,...,...
1719157,920118,2024-06-30,NaN
1719158,920118,2024-07-31,NaN
1719159,920118,2024-08-31,NaN
1719160,920118,2024-09-30,NaN


In [7]:
df2 = df1.dropna()
df2

,证券代码,交易月份,chmom
11,1,2000-12-31,-0.077022
12,1,2001-01-31,-0.122013
13,1,2001-02-28,-0.009110
14,1,2001-03-31,0.028374
15,1,2001-04-30,0.138466
...,...,...,...
1716773,900957,2024-06-30,-0.478696
1716774,900957,2024-07-31,-0.562827
1716775,900957,2024-08-31,-0.809062
1716776,900957,2024-09-30,-0.363842
